# Aggregate Data
This notebook combines the hourly data into daily data. It will also add more columns for later feature extraction

In [27]:
import numpy as np
import pandas as pd

df = pd.read_csv("out.csv")

In [28]:
import datetime as dt
df['DATE'] =  pd.to_datetime(df['DATE'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('DATE')

In [29]:
# make percent attained row
df["dhi_efficiency"] = df["DHI"]/df["Clearsky DHI"]
df["dni_efficiency"] = df["DNI"]/df["Clearsky DNI"]
df["ghi_efficiency"] = df["GHI"]/df["Clearsky GHI"]

In [30]:
grouped = df[df['dhi_efficiency'].notnull()]  
grouped = grouped.groupby('STATION')

daily_summary = pd.DataFrame()

daily_summary['latitude'] = grouped['latitude'].resample('D').last()
daily_summary['longitude'] = grouped['longitude'].resample('D').last()
daily_summary['elevation'] = grouped['elevation'].resample('D').last()

daily_summary['temperature'] = grouped.Temperature.resample('D').mean()
daily_summary['dew_point'] = grouped['Dew Point'].resample('D').mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample('D').mean()
daily_summary['station_pressure'] = grouped.HourlyStationPressure.resample('D').mean()
daily_summary['relative_humidity'] = grouped['Relative Humidity'].resample('D').mean()
daily_summary['wind_direction'] = grouped['Wind Direction'].resample('D').mean()
daily_summary['wind_speed'] = grouped['Wind Speed'].resample('D').mean()
daily_summary['hourly_visibility'] = grouped['HourlyVisibility'].resample('D').mean()
daily_summary['cloud_cover'] = grouped['cloud_cover'].resample('D').mean()

daily_summary['cloud_type'] = grouped['cloud_type'].resample('D').apply(pd.array)
daily_summary['weather_type'] = grouped['weather_type'].resample('D').apply(pd.array)

daily_summary['dhi_efficiency'] = grouped['dhi_efficiency'].resample('D').mean()
daily_summary['dni_efficiency'] = grouped['dni_efficiency'].resample('D').mean()
daily_summary['ghi_efficiency'] = grouped['ghi_efficiency'].resample('D').mean()

In [31]:
import json

cloud_types = []
with open('cloud_lookup.json') as json_file:
    cloud_json = json.load(json_file).values()
    for x in cloud_json:
        cloud_types.append(x['cloud_str'])
    cloud_types = list(dict.fromkeys(cloud_types))
    
with open('weather_lookup_converter.json') as json_file:
    weather_types = list(json.load(json_file).keys())
    
    
# seperate cloud column    
cloud_sep = pd.DataFrame([x for x in daily_summary['cloud_type'].apply(lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)
cloud_sep = cloud_sep.append(pd.DataFrame(columns = cloud_types)).fillna(0.0)
cloud_sep = cloud_sep.loc[:, cloud_sep.columns.notnull()]
cloud_sep = cloud_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(cloud_sep)

# seperate weather column    
weather_sep = pd.DataFrame([x for x in daily_summary['weather_type'].apply(lambda item: dict(map(lambda x: (x, np.count_nonzero(item.to_numpy() == x)), item))).values]).fillna(0)
weather_sep = weather_sep.append(pd.DataFrame(columns = weather_types)).fillna(0.0)
weather_sep = weather_sep.loc[:, weather_sep.columns.notnull()]
weather_sep = weather_sep.set_index(daily_summary.index)
daily_summary = daily_summary.join(weather_sep)

pd.set_option('display.max_columns', None)
daily_summary

latitude  longitude  elevation  temperature  dew_point  \
STATION DATE                                                                 
171     2017-01-01    39.183   -119.733     1432.3     2.100000  -5.100000   
        2017-01-02    39.183   -119.733     1432.3     0.000000  -7.400000   
        2017-01-03    39.183   -119.733     1432.3     2.700000  -1.400000   
        2017-01-04    39.183   -119.733     1432.3     3.700000   0.500000   
        2017-01-05    39.183   -119.733     1432.3    -1.400000  -5.300000   
...                      ...        ...        ...          ...        ...   
279     2017-12-27    39.667   -119.876     1540.2     7.222222  -4.666667   
        2017-12-28    39.667   -119.876     1540.2     8.111111  -2.111111   
        2017-12-29    39.667   -119.876     1540.2     8.555556  -3.222222   
        2017-12-30    39.667   -119.876     1540.2     8.888889  -3.333333   
        2017-12-31    39.667   -119.876     1540.2     8.000000  -5.777778   

                    relative_humidity  station_pressure  wind_direction  \
STATION DATE                                                              
171     2017-01-01          65.804000         24.959000      237.300000   
        2017-01-02          63.226000         25.059000      234.760000   
        2017-01-03          85.507000         25.180000      215.130000   
        2017-01-04          97.122000         25.081000      239.130000   
        2017-01-05          87.982000         25.193000      228.340000   
...                               ...               ...             ...   
279     2017-12-27          45.565556         25.177778      243.122222   
        2017-12-28          52.646667         25.187778      225.211111   
        2017-12-29          45.875556         25.143333      231.344444   
        2017-12-30          45.677778         25.086667      240.966667   
        2017-12-31          42.165556         25.102000      215.200000   

                    wind_speed  hourly_visibility  cloud_cover  \
STATION DATE                                                     
171     2017-01-01    3.800000          10.000000         0.00   
        2017-01-02    3.910000           9.400000         0.54   
        2017-01-03    2.230000          10.000000         0.80   
        2017-01-04    3.120000           3.800000         0.36   
        2017-01-05    1.390000           7.100000         0.94   
...                        ...                ...          ...   
279     2017-12-27    1.077778           9.888889         0.00   
        2017-12-28    1.344444           9.000000         0.00   
        2017-12-29    1.244444          10.000000         0.00   
        2017-12-30    1.466667          10.000000         0.00   
        2017-12-31    0.766667          10.000000         0.00   

                                                           cloud_type  \
STATION DATE                                                            
171     2017-01-01  [clear, clear, clear, clear, clear, clear, cle...   
        2017-01-02  [cloudy, mostly_clear, mostly_cloudy, partly_c...   
        2017-01-03  [mostly_cloudy, mostly_cloudy, mostly_cloudy, ...   
        2017-01-04  [mostly_clear, mostly_clear, mostly_clear, mos...   
        2017-01-05  [mostly_cloudy, mostly_cloudy, cloudy, cloudy,...   
...                                                               ...   
279     2017-12-27  [clear, clear, clear, clear, clear, clear, cle...   
        2017-12-28  [clear, clear, clear, clear, clear, clear, cle...   
        2017-12-29  [clear, clear, clear, clear, clear, clear, cle...   
        2017-12-30  [clear, clear, clear, clear, clear, clear, cle...   
        2017-12-31  [clear, clear, clear, clear, clear, clear, cle...   

                                                         weather_type  \
STATION DATE                                                            
171     2017-01-01  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
 

In [32]:
daily_summary = daily_summary.reset_index(drop=False)
daily_summary = daily_summary.drop(columns=['cloud_type', 'weather_type'])
daily_summary.to_parquet('solar_cleaned.parquet', engine='fastparquet', compression='GZIP')